In [1]:
import numpy as np 
import pandas as pd 
from subprocess import check_output
#print(check_output(["ls", "../input/"]))
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from os.path import join as opj
from matplotlib import pyplot as plt
import tensorflow as tf
import os
%matplotlib inline


In [2]:
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation,Add, ZeroPadding2D ,AveragePooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers,regularizers
from keras.optimizers import Adam,SGD
from keras.initializers import glorot_uniform
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K
K.set_image_data_format('channels_last')
#K.set_learning_phase(1)

Using TensorFlow backend.


In [3]:
train = pd.read_json("input/train.json")
#test = pd.read_json("input/test.json")

In [10]:
only_band1 = ['band1_'+str(i) for i in range(75*75)]
only_band2 = ['band2_'+str(i) for i in range(75*75)]
both_bands = only_band1+only_band2

In [ ]:
# df = pd.concat([pd.DataFrame(test['band_1'].values.tolist(),columns = only_band1),
#                 pd.DataFrame(test['band_2'].values.tolist(),columns = only_band2)],axis = 1)

In [11]:
def collect_test():
    test_data = pd.DataFrame()
    for i in range(4):
        test_piece = pd.read_csv('input/test_subparts/test'+str(i+1)+'.csv')
        test_data = pd.concat([test_data,test_piece])
        test_data = test_data.reset_index().drop(['index'],axis = 1)
    return test_data

In [12]:
test_data = collect_test()

In [13]:
X_band_test_1=np.array(test_data[only_band1]).reshape(8424,75,75)
X_band_test_2=np.array(test_data[only_band2]).reshape(8424,75,75)
X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis], X_band_test_2[:, :, :, np.newaxis], ((X_band_test_1+X_band_test_2)/2)[:, :, :, np.newaxis]], axis=-1)

In [4]:
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis], X_band_2[:, :, :, np.newaxis],((X_band_1+X_band_2)/2)[:, :, :, np.newaxis]], axis=-1)
target_train=train['is_iceberg']

In [ ]:
# X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
# X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
# X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis], X_band_test_2[:, :, :, np.newaxis], ((X_band_test_1+X_band_test_2)/2)[:, :, :, np.newaxis]], axis=-1)

In [ ]:

X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(X_train, target_train, random_state=2,stratify = target_train, train_size=0.80)

### Cross Validation Folds

In [14]:
def data_augmentation(X_input,Y_input,batch_size = 32):
    data_aug = ImageDataGenerator(#featurewise_center=True,
                             #featurewise_std_normalization=True,
                             #zca_whitening=True,
                             rotation_range=0,
                             width_shift_range = 0,
                             height_shift_range = 0,
                             zoom_range = 0,
                             data_format = 'channels_last',
                             horizontal_flip = True,
                             vertical_flip = True)#,fill_mode = 'constant',cval = 0)
    data_aug_batches = data_aug.flow(X_input,Y_input,batch_size = batch_size)
    return data_aug_batches


In [15]:

def cnnmodel():
    X_input = Input(shape = (75,75,3))
    #First composite layer
    X = Conv2D(64,kernel_size = (5,5))(X_input)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(3,3),strides =(2,2))(X)
    X = Dropout(rate = 0.2)(X)
    
    #Second composite layer
    X = Conv2D(128,kernel_size = (3,3))(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(3,3),strides =(2,2))(X)
    X = Dropout(rate = 0.2)(X)
    
    #Third composite layer
    X = Conv2D(128,kernel_size = (3,3))(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(2,2),strides =(2,2))(X)
    X = Dropout(rate = 0.2)(X)

    #Forth composite layer
    X = Conv2D(64,kernel_size = (3,3))(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(3,3),strides =(2,2))(X)
    X = Dropout(rate = 0.2)(X)

    #Flatten layer
    X = Flatten()(X)
    
    #First dense layer
    X = Dense(256,activation = 'relu')(X)
    X = Dropout(rate = 0.2)(X)
   
    #Second dense layer
    X = Dense(256,activation = 'relu')(X)
    X = Dropout(rate = 0.2)(X)
   
    #Decision layer
    X = Dense(1,activation='sigmoid')(X)
    
    model = Model(inputs=X_input,outputs=X)

    return model

def model_compile(model,lr = 0.0001,decay = 0, freezing_layers = None,weights_path = None):
   
    if weights_path:
        model.load_weights(weights_path)
    else:
        pass
    if freezing_layers:
        for layer in model.layers[:freezing_layers]:
            layer.trainable = False
    else:
        pass
    optimizer = Adam(lr = lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=decay)
    #optimizer = SGD(lr, decay=0, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])
    model.summary()
    return model

def get_callbacks(filepath = ".model_weights.hdf5", patience=7, monitor = 'val_loss', verbose = 1, save_best_only = True):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, monitor=monitor, verbose = verbose, save_best_only=True)
    return [es,msave]


In [16]:

def normal_fit(model,callbacks,epochs = 100,batch_size = 32,verbose = 1):
    
    result = model.fit(X_train_cv, y_train_cv,
                          batch_size=batch_size,
                          epochs=epochs,
                          verbose=verbose,
                          validation_data=(X_valid, y_valid),
                          callbacks=callbacks)
    return result

def data_augment_fit(model,callbacks,epochs = 100,batch_size = 32,verbose = 1):
    
    result = model.fit_generator(data_augmentation(X_train_cv,y_train_cv,batch_size = batch_size),
                          steps_per_epoch = len(X_train_cv)/batch_size,
                          epochs=epochs,
                          verbose=verbose,
                          validation_data=(X_valid, y_valid),
                          callbacks=callbacks)
    return result


In [ ]:
Cnnmodel = model_compile(cnnmodel(),lr = 0.0001,decay = 0, freezing_layers = None,weights_path = None)
callbacks = get_callbacks(filepath = 'model save/Model 7-2x filters Advanced CNN with DA and self TL/2018.1.2.hdf5', patience=100, save_best_only = False)
result = normal_fit(Cnnmodel,callbacks = callbacks,epochs = 100,batch_size=32,verbose=2)

In [ ]:
Cnnmodel = model_compile(cnnmodel(),lr = 0.0001,decay = 0, freezing_layers = 5,weights_path = 'model save/Model 7-2x filters Advanced CNN with DA and self TL/2018.1.2.hdf5')
callbacks = get_callbacks(filepath = 'model save/Model 7-2x filters Advanced CNN with DA and self TL/2018.1.2_stage2.hdf5', patience=30, save_best_only = True)
result = data_augment_fit(Cnnmodel,callbacks = callbacks,epochs = 100,batch_size=32,verbose=2)

In [ ]:
Cnnmodel.load_weights('model save/Model 7-2x filters Advanced CNN with DA and self TL/Model-7 cv1_stage1.hdf5')
Cnnmodel.evaluate(X_valid, y_valid)
#Cnnmodel.evaluate(X_train_cv, y_train_cv)

In [ ]:
def save_history(result,name = 'fit_history.csv',path = os.getcwd()):
    train_loss = result.history['loss']
    val_loss = result.history['val_loss']
    train_accuracy = result.history['acc']
    val_accuracy = result.history['val_acc']
    fit_history = pd.DataFrame({'train_loss':train_loss,'val_loss':val_loss,'train_accuracy':train_accuracy,'val_accuracy':val_accuracy})
    fit_history.to_csv(os.path.join(path,name))

In [ ]:
save_history(result_Cnn)

### Prediction

In [8]:
def Model7_CV(X_train,K=3):
    folds = list(StratifiedKFold(n_splits=K, shuffle=True,random_state = 3).split(X_train, target_train))
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, valid_idx) in enumerate(folds):
        print('\n===================FOLD=',j+1)
        global X_train_cv
        X_train_cv = X_train[train_idx]
        global y_train_cv
        y_train_cv = target_train[train_idx]
        global X_valid
        X_valid = X_train[valid_idx]
        global y_valid
        y_valid= target_train[valid_idx]
        
        #Angle
#         X_angle_cv=X_angle[train_idx]
#         X_angle_hold=X_angle[test_idx]

        #define file path and get callbacks
        
        file_path_stage2 = 'model save/Model 9-Advanced CNN with DA(cv5)/Model-9 cv'+str(j+1)+'.hdf5'
        Cnnmodel = model_compile(cnnmodel(),lr = 0.0001,decay = 0, freezing_layers = None,weights_path = None)
        callbacks = get_callbacks(filepath = file_path_stage2, patience=30, save_best_only = True)
        result2 = data_augment_fit(Cnnmodel,callbacks = callbacks,epochs = 150,batch_size=32,verbose=2)
        print('CV'+str(j+1)+', Done!')
#         callbacks = get_callbacks(filepath=file_path, patience=5)
#         gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
#         galaxyModel= getVggAngleModel()
#         galaxyModel.fit_generator(
#                 gen_flow,
#                 steps_per_epoch=24,
#                 epochs=100,
#                 shuffle=True,
#                 verbose=1,
#                 validation_data=([X_holdout,X_angle_hold], Y_holdout),
#                 callbacks=callbacks)

        #Getting the Best Model
        Cnnmodel.load_weights(filepath=file_path_stage2)
        #Getting Training Score
        score = Cnnmodel.evaluate(X_train_cv, y_train_cv)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = Cnnmodel.evaluate(X_valid, y_valid)
        print('Valid loss:', score[0])
        print('Valid accuracy:', score[1])

        #Getting validation Score.
        pred_valid=Cnnmodel.predict(X_valid)
        y_valid_pred_log[valid_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
#         temp_test=Cnnmodel.predict(X_test)
#         y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=Cnnmodel.predict(X_train)
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

#     y_valid_pred_log=y_valid_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_valid_pred_log

In [9]:
Model7_CV(X_train,K = 5)


===================FOLD= 1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 75, 75, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 71, 71, 64)        4864      
_________________________________________________________________
activation_1 (Activation)    (None, 71, 71, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 33, 33, 128)       73856     
_________________________________________________________________
activation_2 (Activation)    (None, 33, 33, 128)

Epoch 24/150
Epoch 00024: val_loss improved from 0.44092 to 0.44000, saving model to model save/Model 9-Advanced CNN with DA(cv5)/Model-9 cv1.hdf5
 - 5s - loss: 0.4140 - acc: 0.8066 - val_loss: 0.4400 - val_acc: 0.7671
Epoch 25/150
Epoch 00025: val_loss improved from 0.44000 to 0.43630, saving model to model save/Model 9-Advanced CNN with DA(cv5)/Model-9 cv1.hdf5
 - 5s - loss: 0.4311 - acc: 0.7936 - val_loss: 0.4363 - val_acc: 0.7888
Epoch 26/150
Epoch 00026: val_loss did not improve
 - 5s - loss: 0.4462 - acc: 0.7921 - val_loss: 0.4810 - val_acc: 0.7919
Epoch 27/150
Epoch 00027: val_loss improved from 0.43630 to 0.39875, saving model to model save/Model 9-Advanced CNN with DA(cv5)/Model-9 cv1.hdf5
 - 5s - loss: 0.4107 - acc: 0.8012 - val_loss: 0.3987 - val_acc: 0.8012
Epoch 28/150
Epoch 00028: val_loss did not improve
 - 5s - loss: 0.4035 - acc: 0.8058 - val_loss: 0.4321 - val_acc: 0.7826
Epoch 29/150
Epoch 00029: val_loss improved from 0.39875 to 0.38005, saving model to model save/M

Epoch 79/150
Epoch 00079: val_loss did not improve
 - 5s - loss: 0.2548 - acc: 0.8810 - val_loss: 0.3018 - val_acc: 0.8292
Epoch 80/150
Epoch 00080: val_loss improved from 0.27509 to 0.26739, saving model to model save/Model 9-Advanced CNN with DA(cv5)/Model-9 cv1.hdf5
 - 5s - loss: 0.2492 - acc: 0.8879 - val_loss: 0.2674 - val_acc: 0.8416
Epoch 81/150
Epoch 00081: val_loss improved from 0.26739 to 0.25776, saving model to model save/Model 9-Advanced CNN with DA(cv5)/Model-9 cv1.hdf5
 - 5s - loss: 0.2361 - acc: 0.8963 - val_loss: 0.2578 - val_acc: 0.8727
Epoch 82/150
Epoch 00082: val_loss did not improve
 - 5s - loss: 0.2343 - acc: 0.8940 - val_loss: 0.3051 - val_acc: 0.8261
Epoch 83/150
Epoch 00083: val_loss did not improve
 - 5s - loss: 0.2337 - acc: 0.8902 - val_loss: 0.2618 - val_acc: 0.8602
Epoch 84/150
Epoch 00084: val_loss did not improve
 - 5s - loss: 0.2332 - acc: 0.8970 - val_loss: 0.2641 - val_acc: 0.8509
Epoch 85/150
Epoch 00085: val_loss did not improve
 - 5s - loss: 0.241

Epoch 140/150
Epoch 00140: val_loss did not improve
 - 5s - loss: 0.1854 - acc: 0.9184 - val_loss: 0.2396 - val_acc: 0.8913
Epoch 141/150
Epoch 00141: val_loss did not improve
 - 5s - loss: 0.2481 - acc: 0.8805 - val_loss: 0.2553 - val_acc: 0.8820
Epoch 142/150
Epoch 00142: val_loss did not improve
 - 5s - loss: 0.1915 - acc: 0.9215 - val_loss: 0.2708 - val_acc: 0.8696
Epoch 143/150
Epoch 00143: val_loss did not improve
 - 5s - loss: 0.1980 - acc: 0.9103 - val_loss: 0.2732 - val_acc: 0.8820
Epoch 144/150
Epoch 00144: val_loss did not improve
 - 5s - loss: 0.2072 - acc: 0.9100 - val_loss: 0.2653 - val_acc: 0.8665
Epoch 145/150
Epoch 00145: val_loss did not improve
 - 5s - loss: 0.1773 - acc: 0.9253 - val_loss: 0.2448 - val_acc: 0.8851
Epoch 146/150
Epoch 00146: val_loss did not improve
 - 5s - loss: 0.1788 - acc: 0.9298 - val_loss: 0.2788 - val_acc: 0.8665
Epoch 147/150
Epoch 00147: val_loss did not improve
 - 5s - loss: 0.1832 - acc: 0.9237 - val_loss: 0.2611 - val_acc: 0.8851
Epoch 14

Epoch 17/150
Epoch 00017: val_loss improved from 0.46198 to 0.43509, saving model to model save/Model 9-Advanced CNN with DA(cv5)/Model-9 cv2.hdf5
 - 5s - loss: 0.4771 - acc: 0.7666 - val_loss: 0.4351 - val_acc: 0.7944
Epoch 18/150
Epoch 00018: val_loss improved from 0.43509 to 0.40623, saving model to model save/Model 9-Advanced CNN with DA(cv5)/Model-9 cv2.hdf5
 - 5s - loss: 0.4684 - acc: 0.7717 - val_loss: 0.4062 - val_acc: 0.8287
Epoch 19/150
Epoch 00019: val_loss did not improve
 - 5s - loss: 0.4716 - acc: 0.7712 - val_loss: 0.4335 - val_acc: 0.7975
Epoch 20/150
Epoch 00020: val_loss improved from 0.40623 to 0.38896, saving model to model save/Model 9-Advanced CNN with DA(cv5)/Model-9 cv2.hdf5
 - 5s - loss: 0.4756 - acc: 0.7782 - val_loss: 0.3890 - val_acc: 0.8318
Epoch 21/150
Epoch 00021: val_loss improved from 0.38896 to 0.37966, saving model to model save/Model 9-Advanced CNN with DA(cv5)/Model-9 cv2.hdf5
 - 5s - loss: 0.4352 - acc: 0.7953 - val_loss: 0.3797 - val_acc: 0.8287
E

...

Epoch 112/150
Epoch 00112: val_loss did not improve
 - 5s - loss: 0.2037 - acc: 0.9207 - val_loss: 0.2267 - val_acc: 0.9031
Epoch 113/150
Epoch 00113: val_loss did not improve
 - 5s - loss: 0.2102 - acc: 0.9115 - val_loss: 0.2158 - val_acc: 0.9031
Epoch 114/150
Epoch 00114: val_loss did not improve
 - 5s - loss: 0.2199 - acc: 0.9039 - val_loss: 0.2157 - val_acc: 0.9094
Epoch 115/150
Epoch 00115: val_loss did not improve
 - 5s - loss: 0.2037 - acc: 0.9161 - val_loss: 0.2250 - val_acc: 0.9000
Epoch 116/150
Epoch 00116: val_loss improved from 0.21555 to 0.21525, saving model to model save/Model 9-Advanced CNN with DA(cv5)/Model-9 cv4.hdf5
 - 5s - loss: 0.1889 - acc: 0.9184 - val_loss: 0.2152 - val_acc: 0.9156
Epoch 117/150
Epoch 00117: val_loss did not improve
 - 5s - loss: 0.2288 - acc: 0.8896 - val_loss: 0.2241 - val_acc: 0.8969
Epoch 118/150
Epoch 00118: val_loss did not improve
 - 5s - loss: 0.1929 - acc: 0.9192 - val_loss: 0.2484 - val_acc: 0.8906
Epoch 119/150
Epoch 00119: val_loss 

Epoch 1/150
Epoch 00001: val_loss improved from inf to 0.68425, saving model to model save/Model 9-Advanced CNN with DA(cv5)/Model-9 cv5.hdf5
 - 7s - loss: 1.1443 - acc: 0.4922 - val_loss: 0.6842 - val_acc: 0.5375
Epoch 2/150
Epoch 00002: val_loss improved from 0.68425 to 0.66739, saving model to model save/Model 9-Advanced CNN with DA(cv5)/Model-9 cv5.hdf5
 - 5s - loss: 0.7414 - acc: 0.5318 - val_loss: 0.6674 - val_acc: 0.6625
Epoch 3/150
Epoch 00003: val_loss improved from 0.66739 to 0.61682, saving model to model save/Model 9-Advanced CNN with DA(cv5)/Model-9 cv5.hdf5
 - 5s - loss: 0.6860 - acc: 0.5564 - val_loss: 0.6168 - val_acc: 0.6062
Epoch 4/150
Epoch 00004: val_loss improved from 0.61682 to 0.57504, saving model to model save/Model 9-Advanced CNN with DA(cv5)/Model-9 cv5.hdf5
 - 5s - loss: 0.6212 - acc: 0.6082 - val_loss: 0.5750 - val_acc: 0.6312
Epoch 5/150
Epoch 00005: val_loss improved from 0.57504 to 0.56133, saving model to model save/Model 9-Advanced CNN with DA(cv5)/Mod

Epoch 44/150
Epoch 00044: val_loss did not improve
 - 5s - loss: 0.2891 - acc: 0.8644 - val_loss: 0.2767 - val_acc: 0.8625
Epoch 45/150
Epoch 00045: val_loss did not improve
 - 5s - loss: 0.2804 - acc: 0.8696 - val_loss: 0.2671 - val_acc: 0.8750
Epoch 46/150
Epoch 00046: val_loss did not improve
 - 5s - loss: 0.2899 - acc: 0.8690 - val_loss: 0.2659 - val_acc: 0.8812
Epoch 47/150
Epoch 00047: val_loss did not improve
 - 5s - loss: 0.2771 - acc: 0.8758 - val_loss: 0.2671 - val_acc: 0.8688
Epoch 48/150
Epoch 00048: val_loss improved from 0.26016 to 0.25779, saving model to model save/Model 9-Advanced CNN with DA(cv5)/Model-9 cv5.hdf5
 - 5s - loss: 0.2703 - acc: 0.8780 - val_loss: 0.2578 - val_acc: 0.8781
Epoch 49/150
Epoch 00049: val_loss did not improve
 - 5s - loss: 0.2826 - acc: 0.8704 - val_loss: 0.2849 - val_acc: 0.8656
Epoch 50/150
Epoch 00050: val_loss improved from 0.25779 to 0.25408, saving model to model save/Model 9-Advanced CNN with DA(cv5)/Model-9 cv5.hdf5
 - 5s - loss: 0.268

Epoch 00102: val_loss did not improve
 - 5s - loss: 0.2222 - acc: 0.9047 - val_loss: 0.3341 - val_acc: 0.8469
Epoch 103/150
Epoch 00103: val_loss improved from 0.22049 to 0.21812, saving model to model save/Model 9-Advanced CNN with DA(cv5)/Model-9 cv5.hdf5
 - 5s - loss: 0.2259 - acc: 0.8972 - val_loss: 0.2181 - val_acc: 0.9125
Epoch 104/150
Epoch 00104: val_loss did not improve
 - 5s - loss: 0.2083 - acc: 0.9032 - val_loss: 0.2237 - val_acc: 0.9125
Epoch 105/150
Epoch 00105: val_loss improved from 0.21812 to 0.21551, saving model to model save/Model 9-Advanced CNN with DA(cv5)/Model-9 cv5.hdf5
 - 5s - loss: 0.2196 - acc: 0.9071 - val_loss: 0.2155 - val_acc: 0.9156
Epoch 106/150
Epoch 00106: val_loss improved from 0.21551 to 0.21315, saving model to model save/Model 9-Advanced CNN with DA(cv5)/Model-9 cv5.hdf5
 - 5s - loss: 0.2094 - acc: 0.8995 - val_loss: 0.2131 - val_acc: 0.9313
Epoch 107/150
Epoch 00107: val_loss did not improve
 - 5s - loss: 0.2056 - acc: 0.9108 - val_loss: 0.2268 

0       4.778432e-03
1       9.378235e-08
2       5.801219e-01
3       2.775462e-03
4       3.599602e-02
5       9.999926e-01
6       9.127076e-01
7       9.055035e-01
8       1.863035e-09
9       2.720955e-02
10      3.456858e-01
11      2.663718e-03
12      7.082685e-01
13      5.788165e-01
14      1.128743e-02
15      5.016693e-02
16      2.937190e-01
17      1.495699e-02
18      2.792191e-01
19      5.917846e-01
20      2.088042e-03
21      1.375084e-01
22      1.131769e-06
23      9.998471e-01
24      3.409592e-08
25      9.998889e-01
26      1.372328e-01
27      3.813254e-04
28      5.572271e-01
29      3.159942e-07
            ...     
1574    3.628112e-01
1575    6.422883e-05
1576    2.578207e-07
1577    1.046593e-01
1578    2.648828e-02
1579    2.473959e-01
1580    5.099518e-02
1581    8.135127e-02
1582    4.655420e-01
1583    4.465244e-01
1584    1.723796e-04
1585    7.843419e-02
1586    4.635813e-01
1587    2.962974e-02
1588    9.974860e-02
1589    2.956966e-03
1590    8.652

### Submission

In [ ]:
def Evaluation_CV(X_train,K=3):
    folds = list(StratifiedKFold(n_splits=K, shuffle=True,random_state = 3).split(X_train, target_train))
    y_test_pred_log=0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, valid_idx) in enumerate(folds):
        print('\n===================FOLD=',j+1)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_valid = X_train[valid_idx]
        y_valid= target_train[valid_idx]
        
        #Angle
#         X_angle_cv=X_angle[train_idx]
#         X_angle_hold=X_angle[test_idx]

        #define file path and get callbacks
        file_path = 'model save/Model 6-Advanced CNN with DA and self TL(cv3)/Model-6 cv'+str(j+1)+'.hdf5'
        Cnnmodel=cnnmodel(input_shape = (75,75,3),lr = 0.0001)
        Cnnmodel.load_weights(file_path)


#         callbacks = get_callbacks(filepath=file_path, patience=5)
#         gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
#         galaxyModel= getVggAngleModel()
#         galaxyModel.fit_generator(
#                 gen_flow,
#                 steps_per_epoch=24,
#                 epochs=100,
#                 shuffle=True,
#                 verbose=1,
#                 validation_data=([X_holdout,X_angle_hold], Y_holdout),
#                 callbacks=callbacks)

        #Getting the Best Model
        #Getting Training Score
        print('cv'+str(j+1))
        score = Cnnmodel.evaluate(X_train_cv, y_train_cv)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = Cnnmodel.evaluate(X_valid, y_valid)
        print('Valid loss:', score[0])
        print('Valid accuracy:', score[1])
        print('\n')

        #Getting validation Score.
        pred_valid=Cnnmodel.predict(X_valid)
        y_valid_pred_log[valid_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=Cnnmodel.predict(X_test)
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=Cnnmodel.predict(X_train)
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_valid_pred_log=y_valid_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log

In [ ]:
preds = Evaluation_CV(X_train,K=5)
sub = test[['id']]
sub['is_iceberg'] = preds
sub.to_csv('5cv.csv',index = False)

### Submission(min max median stacking)

In [17]:
def test_scores(K=5):
    y_test_pred_log=0
    test_df = test_data[['id']]
    for j in range(K):
        print('\n===================FOLD=',j+1)
        file_path = 'model save/Model 9-Advanced CNN with DA(cv5)/Model-9 cv'+str(j+1)+'.hdf5'
        Cnnmodel = model_compile(cnnmodel(),lr = 0.0001,decay = 0, freezing_layers = None,weights_path = file_path)

        #Getting Test Scores
        temp_test=Cnnmodel.predict(X_test)
        test_df['cv'+str(j+1)]=temp_test.reshape(temp_test.shape[0])
    return test_df

In [18]:
test_df = test_scores()


===================FOLD= 1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 75, 75, 3)         0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 71, 71, 64)        4864      
_________________________________________________________________
activation_21 (Activation)   (None, 71, 71, 64)        0         
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 35, 35, 64)        0         
_________________________________________________________________
dropout_31 (Dropout)         (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 33, 33, 128)       73856     
_________________________________________________________________
activation_22 (Activation)   (None, 33, 33, 128)

c:\users\yltbe\anaconda3\envs\tensorflow_gpu\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()



===================FOLD= 2
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 75, 75, 3)         0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 71, 71, 64)        4864      
_________________________________________________________________
activation_25 (Activation)   (None, 71, 71, 64)        0         
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 35, 35, 64)        0         
_________________________________________________________________
dropout_37 (Dropout)         (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 33, 33, 128)       73856     
_________________________________________________________________
activation_26 (Activation)   (None, 33, 33, 128)


===================FOLD= 5
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 75, 75, 3)         0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 71, 71, 64)        4864      
_________________________________________________________________
activation_37 (Activation)   (None, 71, 71, 64)        0         
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 35, 35, 64)        0         
_________________________________________________________________
dropout_55 (Dropout)         (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 33, 33, 128)       73856     
_________________________________________________________________
activation_38 (Activation)   (None, 33, 33, 128)

In [19]:
test_df

,id,cv1,cv2,cv3,cv4,cv5
0,5941774d,6.024951e-02,6.976975e-03,1.599942e-01,3.067267e-02,3.822886e-02
1,4023181e,4.296758e-01,2.636851e-01,3.893755e-01,3.061959e-01,3.619502e-01
2,b20200e4,2.948733e-06,7.470820e-09,8.316091e-08,1.818759e-10,9.530052e-07
3,e7f018bb,9.999977e-01,9.972776e-01,9.943120e-01,9.999818e-01,9.999874e-01
4,4371c8c3,1.594899e-02,2.674861e-03,5.769277e-02,5.403951e-03,8.309172e-03
5,a8d9b1fd,7.127362e-02,3.318478e-02,4.162203e-02,1.362492e-02,1.232468e-01
6,29e7727e,1.399796e-01,7.882445e-02,8.054397e-02,4.562308e-02,2.718464e-01
7,92a51ffb,9.999316e-01,9.998859e-01,9.892173e-01,9.997003e-01,9.994119e-01
8,c769ac97,2.110788e-06,1.501543e-04,2.666947e-05,8.447725e-07,7.176145e-05
9,aee0547d,1.601085e-09,2.687179e-07,1.855354e-07,1.429564e-09,7.114147e-07


In [20]:
def get_sub_mmm(test_df):
    prediction = test_df
    prediction['is_iceberg'] = np.where(np.all(prediction[prediction.columns[1:]]>0.9,axis = 1),
                                    np.max(prediction[prediction.columns[1:]],axis = 1),
                                     np.where(np.all(prediction[prediction.columns[1:]]<0.1,axis = 1),np.min(prediction[prediction.columns[1:]],axis = 1),np.median(prediction[prediction.columns[1:]],axis = 1)))
    sub = prediction[['id','is_iceberg']]
    sub.to_csv('sub.csv',index = False)
    return sub

In [21]:
sub = get_sub_mmm(test_df)

c:\users\yltbe\anaconda3\envs\tensorflow_gpu\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [22]:
sub

,id,is_iceberg
0,5941774d,3.822886e-02
1,4023181e,3.619502e-01
2,b20200e4,1.818759e-10
3,e7f018bb,9.999977e-01
4,4371c8c3,2.674861e-03
5,a8d9b1fd,4.162203e-02
6,29e7727e,8.054397e-02
7,92a51ffb,9.999316e-01
8,c769ac97,8.447725e-07
9,aee0547d,1.429564e-09
